In [1]:
import pandas as pd
import os

#!pip install --upgrade pip
#!pip install google
#!pip install google-cloud
#!pip install google-cloud-storage
#!pip install google-cloud-bigquery
!pip install db-dtypes

In [5]:
from google.cloud import bigquery
import pandas as np
import numpy as np


/Users/titouan/.pyenv/versions/3.10.6/envs/monitor-reactor/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
GCP_MONITOR_THE_REACTOR = 'monitor-the-reactor'

RELATIVE_PATH = 'data/raw_data'

BQ_FAULTFREE_TRAIN = 'TEP_FaultFree_training'
BQ_FAULTFREE_TEST = 'TEP_FaultFree_testing'
BQ_FAULTY_TRAIN = 'TEP_Faulty_Training'
BQ_FAULTY_TEST = 'TEP_Faulty_testing'

COLUMNS_NAMES = [
       'faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2',
       'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11'
]

COLUMNS_TO_KEEP = (COLUMNS_NAMES)
COLUMNS_TO_DROP = tuple(set(COLUMNS_NAMES) - set())


SAMPLE_DIVISION = 10

In [3]:
def get_data(
    project_id=GCP_MONITOR_THE_REACTOR,
    dataset=BQ_FAULTY_TRAIN,
    col_to_keep=0,
    col_to_drop=0,
    sample_division=SAMPLE_DIVISION,
) :

    if col_to_keep == 0:
        query = f"""
        SELECT {', '.join(col_to_drop)}
        FROM `{project_id}`.`{dataset}`.`csv`
        WHERE
        MOD(sample, {sample_division}) = 0
        ORDER BY faultNumber, simulationRun, sample
        """

        client = bigquery.Client(project=project_id, location="EU")
        query_job = client.query(query)
        result = query_job.result()
        df = result.to_dataframe()

        return df

    elif col_to_drop == 0:
        query = f"""
        SELECT {', '.join(col_to_keep)}
        FROM `{project_id}`.`{dataset}`.`csv`
        WHERE
        MOD(sample, {sample_division}) = 0
        ORDER BY faultNumber, simulationRun, sample
        """

        client = bigquery.Client(project=project_id, location="EU")
        query_job = client.query(query)
        result = query_job.result()
        df = result.to_dataframe()

        return df

In [6]:
get_data(col_to_keep=COLUMNS_TO_KEEP)

/Users/titouan/.pyenv/versions/3.10.6/envs/monitor-reactor/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,1,1,10,0.29372,3675.9,4487.4,9.4107,26.809,42.144,2705.3,...,53.692,28.741,61.490,22.409,39.615,38.309,47.296,47.513,42.355,18.957
1,1,1,20,0.25299,3650.4,4477.0,9.2933,27.165,42.307,2699.2,...,54.040,24.512,61.572,22.608,39.999,35.128,45.325,46.273,41.153,18.022
2,1,1,30,0.32790,3672.1,4498.5,9.3309,26.882,42.230,2731.1,...,54.306,31.892,61.533,19.979,40.044,37.061,50.021,48.708,40.319,17.414
3,1,1,40,0.57525,3603.4,4487.5,9.1370,26.848,42.636,2783.8,...,54.260,56.716,60.736,20.244,38.633,33.167,49.407,49.782,40.344,17.442
4,1,1,50,0.70478,3682.0,4521.7,8.9201,27.062,42.055,2806.5,...,54.007,69.024,54.835,21.451,32.382,38.387,47.022,50.995,40.224,19.915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,20,500,460,0.20371,3718.2,4540.1,9.3886,26.788,42.179,2723.9,...,54.483,19.862,59.980,17.811,38.043,34.616,46.801,43.015,40.696,20.419
499996,20,500,470,0.25271,3656.2,4541.4,9.2657,26.660,42.457,2712.2,...,54.350,25.266,60.311,21.856,38.525,41.560,47.386,43.635,41.684,17.655
499997,20,500,480,0.28501,3617.1,4461.2,9.3182,26.752,42.570,2709.8,...,54.374,27.663,60.057,21.871,36.417,35.125,51.819,44.903,41.511,19.924
499998,20,500,490,0.26098,3689.1,4500.3,9.2441,26.698,42.348,2704.2,...,54.567,25.395,60.380,21.318,38.924,37.029,48.305,46.146,41.055,16.651


In [ ]:
print(COLUMNS_TO_KEEP)
print(COLUMNS_TO_DROP)
print(query)


['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11']
('xmeas_41', 'xmeas_14', 'xmeas_34', 'xmeas_2', 'xmeas_15', 'xmv_3', 'xmv_4', 'xmeas_38', 'xmeas_36', 'xmeas_33', 'simulationRun', 'xmeas_5', 'xmeas_30', 'xmeas_32', 'xmeas_19', 'xmv_7', 'xmeas_8', 'xmv_11', 'xmv_9', 'xmeas_22', 'xmeas_35', 'xmeas_1', 'xmeas_25', 'xmeas_28', 'xmeas_40', 'xmv_8', 'sample', 'xmeas_37', 'xmeas_21', 'xmeas_11', 'xmeas_26', 'xmeas_17', 'xmeas